In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

import yaml
import pandas as pd

from src.data.load import load_data
from src.data.prepare import prepare_data
from src.models.cv_iterator import leave_last_k

In [2]:
# read config
with open('../config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

In [3]:
# load and prepare data
dfs = load_data(config=config['data_loader'])
#dfs = prepare_data(dataframes=dfs)

In [4]:
# train-test split
df_train, df_test = leave_last_k(df=dfs['data'], config=config['optimization'])
del df_test

In [5]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

df = df_train.groupby('user_id')['item_id'].apply(list).tolist()
te = TransactionEncoder()
df = te.fit(df).transform(df)
df = pd.DataFrame(df, columns=te.columns_)

frequent_itemsets = fpgrowth(df, min_support=0.15, use_colnames=True)
del df
frequent_itemsets

,support,itemsets
0,0.584305,(50)
1,0.524920,(100)
2,0.516437,(258)
3,0.513256,(181)
4,0.457052,(1)
...,...,...
29550,0.152704,"(210, 732)"
29551,0.151644,"(202, 732)"
29552,0.151644,"(172, 732)"
29553,0.150583,"(50, 732, 174)"


In [6]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(50),(100),0.584305,0.524920,0.396607,0.678766,1.293083,1.0,0.089893,1.478920,0.545243,0.556548,0.323831,0.717161
1,(100),(50),0.524920,0.584305,0.396607,0.755556,1.293083,1.0,0.089893,1.700569,0.477088,0.556548,0.411961,0.717161
2,"(258, 100)",(50),0.271474,0.584305,0.225875,0.832031,1.423966,1.0,0.067251,2.474833,0.408683,0.358586,0.595932,0.609301
3,"(258, 50)",(100),0.324496,0.524920,0.225875,0.696078,1.326065,1.0,0.055540,1.563165,0.364008,0.362245,0.360272,0.563191
4,(100),"(258, 50)",0.524920,0.324496,0.225875,0.430303,1.326065,1.0,0.055540,1.185725,0.517574,0.362245,0.156634,0.563191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580447,"(50, 181)",(732),0.475080,0.186638,0.151644,0.319196,1.710240,1.0,0.062976,1.194708,0.791142,0.297297,0.162976,0.565848
580448,"(732, 181)",(50),0.155885,0.584305,0.151644,0.972789,1.664864,1.0,0.060559,15.276776,0.473100,0.257658,0.934541,0.616159
580449,(50),"(732, 181)",0.584305,0.155885,0.151644,0.259528,1.664864,1.0,0.060559,1.139968,0.960682,0.257658,0.122783,0.616159
580450,(732),"(50, 181)",0.186638,0.475080,0.151644,0.812500,1.710240,1.0,0.062976,2.799576,0.510581,0.297297,0.642803,0.565848
